# Домашнее задание №5.  Part-of-Speech разметка, NER, извлечение отношений

In [742]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#для задания 1
import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

#для задания 2
import os
import corus
#!pip install -U spacy
#!python -m spacy info
#!python -m spacy download ru_core_news_sm
import spacy
from spacy import displacy
import ru_core_news_sm
#!pip install slovnet
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

## Задание
**Задание 1.** Написать теггер на данных с русским языком

-- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации;

-- написать свой теггер как на занятии (попробовать разные векторайзеры, добавить знание не только букв, но и слов);

-- сравнить все реализованные методы сделать выводы.
 
**Задание 2.** Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/

-- проверить NER из nltk/spacy/deeppavlov

-- написать свой нер попробовать разные подходы:

передаём в сетку токен и его соседей

передаём в сетку только токен

-- сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


## План решения

[1. POS tagging на русскоязычном корпусе](#section_1)

[1.1. Теггеры UnigramTagger, BigramTagger, TrigramTagger и их комбмнации](#section_1.1)

[1.2. Самописный теггер с различными векторайзерами](#section_1.2)

[1.3. Выводы](#section_1.3)

[2. Named-entity recognition (NER)](#section_2)

[2.1. NER из NLTK](#section_2.1)

[2.2. NER из Spacy](#section_2.2)

[2.3. Ner из slovnet](#section_2.3)

[2.4. Самописный NER](#section_2.4)

[2.5. Выводы](#section_2.5)

### 1. POS tagging на русскоязычном корпусе  <a id='section_1'></a>

Данные были предварительно скачены на локальный компьютер с https://github.com/UniversalDependencies/UD_Russian-SynTagRus.

Загружаем данные

In [269]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [270]:
#просматриваем 2 предложения тренировочных данных:
#в каждом из них для каждого токена выводим form (словоформа для чтения) и upos (часть речи)
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



Собираем данные

In [272]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [273]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [274]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [275]:
#просматриваем тренировочные данные (токен, тег)
fdata_train[:5]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

### 1.1. Теггеры UnigramTagger, BigramTagger, TrigramTagger и их комбмнации <a id='section_1.1'></a>

**UnigramTagger** (
UnigramTagger учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова)

In [276]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_U = unigram_tagger.evaluate(fdata_test)
display(accuracy_U)

0.823732013802982

**BigramTagger** (BigramTagger учитывает тэги двух слов: текущее и предыдущее слово)

In [277]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_B = bigram_tagger.evaluate(fdata_test)
display(accuracy_B)

0.6093886320724006

**TrigramTagger**

In [278]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_T = trigram_tagger.evaluate(fdata_test)
display(accuracy_T)

0.1778631421316492

**Комбинация теггеров**

In [279]:
list_1 = [UnigramTagger, BigramTagger]

list_2 = [UnigramTagger, TrigramTagger]

list_3 = [BigramTagger, TrigramTagger]

list_4 = [UnigramTagger, BigramTagger, TrigramTagger]

accuracy_N = []

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

for list_N in [list_1, list_2, list_3, list_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         list_N,  
                         backoff = backoff) 

    accuracy_N.append(tag.evaluate(fdata_test))

accuracy_N

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [280]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 'UnigramTagger+BigramTagger+TrigramTagger'], 'Accuracy' : [accuracy_U, accuracy_B, accuracy_T] + accuracy_N})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


**Вывод:** лучшие показатели метрики у комбинации теггеров UnigramTagger+BigramTagger+TrigramTagger

### 1.2. Самописный теггер с различными векторайзерами <a id='section_1.2'></a>

In [282]:
#собираем токены и лэйблы в списки
train_tok = [] #список токенов
train_label = [] #список теггов
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [283]:
#кодируем целевые метки значениями от 0 до (кол-во классов - 1)
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [284]:
#строим модели логистической регрессии с различными векторайзерами
#P.S. тестовые данные срезаны до 115000 штук из-за битых файлов

vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [285]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921158
9,"HashingVectorizer(analyzer='char', n_features=...",0.903654
8,"HashingVectorizer(analyzer='char', n_features=...",0.901213
7,"HashingVectorizer(analyzer='char', n_features=...",0.897075
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882215
4,"TfidfVectorizer(ngram_range=(1, 3))",0.662784
3,"CountVectorizer(ngram_range=(1, 3))",0.658415
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.620760


In [286]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923583
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905009
7,"HashingVectorizer(analyzer='char', n_features=...",0.900913
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885157
4,"TfidfVectorizer(ngram_range=(1, 3))",0.653252
3,"CountVectorizer(ngram_range=(1, 3))",0.649052
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.630539


**Вывод:** лучшие показатели метрик f1 и accuracy у CountVectorizer(analyzer='char', ngram_range=(1, 3))

### 1.3. Выводы <a id='section_1.3'></a>

In [287]:
result.sort_values('Accuracy', ascending=False).head(1)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905


In [288]:
result_model.sort_values('f1_score', ascending=False).head(1)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836


In [289]:
result_model_acc.sort_values('Accuracy', ascending=False).head(1)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513


**Вывод:** лучший метод pos tagging (по результатам подсчета метрик) -- самописный теггер с CountVectorizer(analyzer='char', ngram_range=(1, 3))

### 2. Named-entity recognition (NER) <a id='section_2'></a>

Рассмотрим задачу извлечения именованных сущностей на основе размеченной коллекции http://www.labinform.ru/pub/named_entities/collection3.

Датасет содержит 1000 файлов txt и 1000 файлов ann, в которых размечены 3 сущности:
 - имена людей (PER),
 - имена организаций (ORG),
 - географические названия (LOC). 
 
 P.S. Все файлы предварительно загружены на локальный компьютер; файлы 595.txt и 595.ann битые, в следcтвие чего были удалены из рассмотрения.

**Проверим работу NER из nltk, NER из spacy и NER из slovnet на указанной коллекции**

**Важно:** загрузка файлов через load_ne5(dir) невозможна из-за ошибки кодировки при открытии файлов (UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 171: character maps to <undefined>)

P.S. deeppavlov не рассматривается в виду отсутствия необходимых вычислительных мощностей для его установки (надо переустанавливать tensorflow на более раннюю версию).

In [718]:
#просматриваем содержание коллекции
print(os.listdir("collection3"))

['001.ann', '001.txt', '002.ann', '002.txt', '003.ann', '003.txt', '004.ann', '004.txt', '005.ann', '005.txt', '006.ann', '006.txt', '007.ann', '007.txt', '008.ann', '008.txt', '009.ann', '009.txt', '010.ann', '010.txt', '011.ann', '011.txt', '012.ann', '012.txt', '013.ann', '013.txt', '014.ann', '014.txt', '015 (!).ann', '015 (!).txt', '016.ann', '016.txt', '017.ann', '017.txt', '018.ann', '018.txt', '019.ann', '019.txt', '020.ann', '020.txt', '021.ann', '021.txt', '022.ann', '022.txt', '023.ann', '023.txt', '025.ann', '025.txt', '026.ann', '026.txt', '027.ann', '027.txt', '028.ann', '028.txt', '029.ann', '029.txt', '030.ann', '030.txt', '031.ann', '031.txt', '032.ann', '032.txt', '033.ann', '033.txt', '034.ann', '034.txt', '035.ann', '035.txt', '036.ann', '036.txt', '037.ann', '037.txt', '038.ann', '038.txt', '039.ann', '039.txt', '03_12_12a.ann', '03_12_12a.txt', '03_12_12b.ann', '03_12_12b.txt', '03_12_12c.ann', '03_12_12c.txt', '03_12_12d.ann', '03_12_12d.txt', '03_12_12g.ann', '0

In [719]:
#собираем только текстовые файлы коллекции
fileDir = r"collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_txt)

['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015 (!).txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '03_12_12a.txt', '03_12_12b.txt', '03_12_12c.txt', '03_12_12d.txt', '03_12_12g.txt', '03_12_12h.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '04_02_13a_abdulatipov.txt', '04_03_13a_sorokin.txt', '04_12_12b.txt', '04_12_12d.txt', '04_12_12f.txt', '04_12_12g.txt', '04_12_12h_corr.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '

In [720]:
#заносим данные файлов txt в датасет
text_list = []
for file in documents_txt:
    doc = open('collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...
...,...
994,"Депутат от ""ЕР"": К отставке А.Сердюкова причас..."
995,\nСи Цзиньпин избран генсеком Коммунистической...
996,"""Ведомости"" узнали о смене лидера московских е..."
997,СМИ узнали о кутежах туркменского чиновника на...


### 2.1. NER из NLTK <a id='section_2.1'></a>

In [721]:
#требуется для токенизации
nltk.download('punkt')
#требуется для parts of speech tagging
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BazhanovaEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\BazhanovaEN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [722]:
#пример текста
document = data_text.text[0]

#разбиваем документ на токены и применяем pos tagging (на выходе список кортежей (токен, часть речи))
nltk.pos_tag(nltk.word_tokenize(document))

[('Россия', 'JJ'),
 ('рассчитывает', 'NNP'),
 ('на', 'NNP'),
 ('конструктивное', 'NNP'),
 ('воздействие', 'NNP'),
 ('США', 'NNP'),
 ('на', 'NNP'),
 ('Грузию', 'VBD'),
 ('04/08/2008', 'CD'),
 ('12:08', 'CD'),
 ('МОСКВА', 'NN'),
 (',', ','),
 ('4', 'CD'),
 ('авг', 'SYM'),
 ('-', ':'),
 ('РИА', 'NN'),
 ('Новости', 'NN'),
 ('.', '.'),
 ('Россия', 'JJ'),
 ('рассчитывает', 'NN'),
 (',', ','),
 ('что', 'NNP'),
 ('США', 'NNP'),
 ('воздействуют', 'NNP'),
 ('на', 'NNP'),
 ('Тбилиси', 'NNP'),
 ('в', 'NNP'),
 ('связи', 'NNP'),
 ('с', 'NNP'),
 ('обострением', 'NNP'),
 ('ситуации', 'NNP'),
 ('в', 'NNP'),
 ('зоне', 'NNP'),
 ('грузино-осетинского', 'JJ'),
 ('конфликта', 'NNP'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('статс-секретарь', 'JJ'),
 ('-', ':'),
 ('заместитель', 'NN'),
 ('министра', 'JJ'),
 ('иностранных', 'NNP'),
 ('дел', 'NNP'),
 ('России', 'NNP'),
 ('Григорий', 'NNP'),
 ('Карасин', 'NNP'),
 ('заявил', 'NNP'),
 ('в', 'NNP'),
 ('телефонном', 'NNP'),
 ('разговоре', 'NNP'),
 ('с', 'NNP

In [723]:
#распознаем именнованные сущности с помощью классификатора (Person, Organization, GPE)
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

In [724]:
# разметка из collection3
pd.read_csv('collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


**Вывод:** многие примеры сущностей NER из nltk не обнаружила.

### 2.2. NER из Spacy <a id='section_2.2'></a>

In [725]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [726]:
#список токенов, частей речи и сущностей
for token in article:
    print(token.text, token.pos_, token.dep_)

Россия PROPN nsubj
рассчитывает VERB ROOT
на ADP case
конструктивное ADJ amod
воздействие NOUN obl
США PROPN nmod
на ADP case
Грузию PROPN nmod


 SPACE dep
04/08/2008 NUM flat:foreign
12:08 NUM appos


 SPACE dep
МОСКВА PROPN obj
, PUNCT punct
4 NUM nummod:gov
авг NOUN flat
- NOUN conj
РИА PROPN conj
Новости PROPN appos
. PUNCT punct
Россия PROPN nsubj
рассчитывает VERB ROOT
, PUNCT punct
что SCONJ mark
США PROPN nsubj
воздействуют VERB ccomp
на ADP case
Тбилиси PROPN obl
в ADP case
связи NOUN fixed
с ADP fixed
обострением NOUN obl
ситуации NOUN nmod
в ADP case
зоне NOUN nmod
грузино ADJ amod
- ADJ amod
осетинского ADJ amod
конфликта NOUN nmod
. PUNCT punct
Об ADP case
этом PRON obl
статс NOUN nsubj
- NOUN nsubj
секретарь NOUN nsubj
- NOUN nsubj
заместитель NOUN nsubj
министра NOUN nmod
иностранных ADJ amod
дел NOUN nmod
России PROPN nmod
Григорий PROPN appos
Карасин PROPN flat:name
заявил VERB ROOT
в ADP case
телефонном ADJ amod
разговоре NOUN obl
с ADP case
заместителем NOUN nmod
го

In [727]:
# разметка из collection3
pd.read_csv('collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


**Вывод:** NER из spacy нашла все сущности.

### 2.3. NER из slovnet <a id='section_2.3'></a>

In [729]:
text = data_text.text[0]
navec = Navec.load('slovnet/navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet/slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Россия рассчитывает на конструктивное воздействие США на Грузию
LOC───                                            LOC    LOC───
04/08/2008 12:08
МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют
LOC───          ORG────────  LOC───                   LOC             
 на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского 
    LOC────                                                           
конфликта. Об этом статс-секретарь - заместитель министра иностранных 
дел России Григорий Карасин заявил в телефонном разговоре с 
    LOC─── PER─────────────                                 
заместителем госсекретаря США Дэниэлом Фридом.
                          LOC PER──────────── 
"С российской стороны выражена глубокая озабоченность в связи с новым 
витком напряженности вокруг Южной Осетии, противозаконными действиями 
                            LOC─────────                              
грузинской стороны по наращиванию своих вооруженных сил в регионе, 
бес

**Вывод:** NER из slovnet работает также хорошо, как NER из spacy (нашел все сущности).

### 2.4. Самописный NER <a id='section_2.4'></a>

In [730]:
#собираем только ann файлы коллекции
fileDir = r"collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['001.ann', '002.ann', '003.ann', '004.ann', '005.ann', '006.ann', '007.ann', '008.ann', '009.ann', '010.ann', '011.ann', '012.ann', '013.ann', '014.ann', '015 (!).ann', '016.ann', '017.ann', '018.ann', '019.ann', '020.ann', '021.ann', '022.ann', '023.ann', '025.ann', '026.ann', '027.ann', '028.ann', '029.ann', '030.ann', '031.ann', '032.ann', '033.ann', '034.ann', '035.ann', '036.ann', '037.ann', '038.ann', '039.ann', '03_12_12a.ann', '03_12_12b.ann', '03_12_12c.ann', '03_12_12d.ann', '03_12_12g.ann', '03_12_12h.ann', '040.ann', '041.ann', '042.ann', '043.ann', '044.ann', '045.ann', '046.ann', '047.ann', '048.ann', '049.ann', '04_02_13a_abdulatipov.ann', '04_03_13a_sorokin.ann', '04_12_12b.ann', '04_12_12d.ann', '04_12_12f.ann', '04_12_12g.ann', '04_12_12h_corr.ann', '050.ann', '051.ann', '052.ann', '053.ann', '054.ann', '055.ann', '056.ann', '057.ann', '058.ann', '059.ann', '060.ann', '061.ann', '062.ann', '063.ann', '064.ann', '065.ann', '066.ann', '067.ann', '068.ann', '069.ann', '

In [731]:
ann = pd.read_csv('collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [732]:
#составляем списки токенов и интенсов (из файла .ann делается словарь {слово : интенс}, из словаря каждому токену сопоствляем интенс)
docs = []
for i in range(len(data_text)):
    words = []
    labels = []
    #подготавливаем текст
    text = data_text['text'][i]
    
    df = pd.read_csv('collection3/' + documents_ann[i], delimiter='\t', header=None)
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [733]:
print(docs[0][0]), print(docs[0][1])

['Россия', 'рассчитывает', 'на', 'конструктивное', 'воздействие', 'США', 'на', 'Грузию', '04/08/2008', '12', ':', '08', 'МОСКВА', ',', '4', 'авг', '-', 'РИА', 'Новости', '.', 'Россия', 'рассчитывает', ',', 'что', 'США', 'воздействуют', 'на', 'Тбилиси', 'в', 'связи', 'с', 'обострением', 'ситуации', 'в', 'зоне', 'грузино-осетинского', 'конфликта', '.', 'Об', 'этом', 'статс-секретарь', '-', 'заместитель', 'министра', 'иностранных', 'дел', 'России', 'Григорий', 'Карасин', 'заявил', 'в', 'телефонном', 'разговоре', 'с', 'заместителем', 'госсекретаря', 'США', 'Дэниэлом', 'Фридом', '.', '"', 'С', 'российской', 'стороны', 'выражена', 'глубокая', 'озабоченность', 'в', 'связи', 'с', 'новым', 'витком', 'напряженности', 'вокруг', 'Южной', 'Осетии', ',', 'противозаконными', 'действиями', 'грузинской', 'стороны', 'по', 'наращиванию', 'своих', 'вооруженных', 'сил', 'в', 'регионе', ',', 'бесконтрольным', 'строительством', 'фортификационных', 'сооружений', '"', ',', '-', 'говорится', 'в', 'сообщении', '

(None, None)

In [734]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Россия	LOC
рассчитывает	OUT
на	OUT
конструктивное	OUT
воздействие	OUT
США	LOC
на	OUT
Грузию	LOC
04/08/2008	OUT
12	OUT
:	OUT
08	OUT
МОСКВА	LOC
,	OUT
4	OUT
авг	OUT
-	OUT
РИА	ORG
Новости	ORG
.	OUT
Россия	LOC
рассчитывает	OUT
,	OUT
что	OUT
США	LOC
воздействуют	OUT
на	OUT
Тбилиси	LOC
в	OUT
связи	OUT
с	OUT
обострением	OUT
ситуации	OUT
в	OUT
зоне	OUT
грузино-осетинского	OUT
конфликта	OUT
.	OUT
Об	OUT
этом	OUT
статс-секретарь	OUT
-	OUT
заместитель	OUT
министра	OUT
иностранных	OUT
дел	OUT
России	LOC
Григорий	PER
Карасин	PER
заявил	OUT
в	OUT
телефонном	OUT
разговоре	OUT
с	OUT
заместителем	OUT
госсекретаря	OUT
США	LOC
Дэниэлом	PER
Фридом	PER
.	OUT
"	OUT
С	OUT
российской	OUT
стороны	OUT
выражена	OUT
глубокая	OUT
озабоченность	OUT
в	OUT
связи	OUT
с	OUT
новым	OUT
витком	OUT
напряженности	OUT
вокруг	OUT
Южной	LOC
Осетии	LOC
,	OUT
противозаконными	OUT
действиями	OUT
грузинской	OUT
стороны	OUT
по	OUT
наращиванию	OUT
своих	OUT
вооруженных	OUT
сил	OUT
в	OUT
регионе	OUT
,	OUT
бесконтрольным	OUT
строительс

Оболочка **sklearn_crfsuite**

In [735]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Россия,LOC
1,0,рассчитывает,OUT
2,0,на,OUT
3,0,конструктивное,OUT
4,0,воздействие,OUT
5,0,США,LOC
6,0,на,OUT
7,0,Грузию,LOC
8,0,04/08/2008,OUT
9,0,12,OUT


**Будем передавать в модель токен с соседями**

In [736]:
#преобразуем датасет: 
#группируем записи по номеру предложения (аггфункция - список кортежей (токен, лейбл))
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

#преобразует датасет df
getter = SentenceGetter(df)
#получаем список номеров преложений датасета df
sentences = getter.sentences

In [737]:
#преобразуем слова в признаки:
#для каждого слова-токена составляем словарь признаков, 
#определяем является ли оно начаотным, конечным в предложении
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

#преобразуем предложения:
#каждое слово предложения преобразуем в признаки
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

#список всех лейблов предложения
def sent2labels(sent):
    return [label for token, label in sent]

#список всех токенов предложения
def sent2tokens(sent):
    return [token for token, label in sent]

In [738]:
#собираем данные и метки
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [739]:
#передаем в датасет токен с соседями (первый индекс -- предложение, 2 индекс -- слово)
X[2][7]

{'bias': 1.0,
 'word.lower()': 'в',
 'word[-3:]': 'в',
 'word[-2:]': 'в',
 'word.isdigit()': False,
 '-1:word.lower()': 'изъяты',
 '+1:word.lower()': 'арендуемом'}

In [740]:
len(X)

999

In [741]:
#разбиваем данные на train и test
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [747]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Градиентный спуск с использованием метода L-BFGS
    c1=0.1, #Коэффициент для регуляризации L1
    c2=0.1, #Коэффициент для регуляризации L2
    max_iterations=1000, #Максимальное количество итераций
    all_possible_transitions=True, #Генерация объектов (не встречающихся в обучающих данных)
    verbose=True #Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|████████████████████████████████████████████| 700/700 [00:01<00:00, 579.48it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 98972
Seconds required: 0.344

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.12  loss=121356.43 active=98097 feature_norm=1.00
Iter 2   time=0.06  loss=108933.37 active=94193 feature_norm=1.59
Iter 3   time=0.06  loss=106719.02 active=97471 feature_norm=1.47
Iter 4   time=0.06  loss=105586.47 active=97173 feature_norm=1.40
Iter 5   time=0.06  loss=103864.92 active=97820 feature_norm=1.46
Iter 6   time=0.06  loss=88408.59 active=97567 feature_norm=4.93
Iter 7   time=0.06  loss=86020.46 active=98489 feature_norm=4.83
Iter 8   time=0.06  loss=82185.93 active=98955 feature_norm=5.16
Iter 9   time=0.11  loss=78184.62 active=98904 feature_norm=6.24
Iter 10 

In [748]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [749]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities) #f1 для элементов последовательности

0.9464624149713488

In [750]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

              precision    recall  f1-score   support

         LOC       0.94      0.80      0.86      2613
         ORG       0.83      0.62      0.71      4260
         OUT       0.96      0.99      0.97     66116
         PER       0.92      0.79      0.85      4740

    accuracy                           0.95     77729
   macro avg       0.91      0.80      0.85     77729
weighted avg       0.95      0.95      0.95     77729



**Будем передавать в модель только токен**

In [751]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [752]:
#собираем данные и метки
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [754]:
#передаем в датасет только токен
X[1][7]

{'bias': 1.0,
 'word.lower()': 'беженцев',
 'word[-3:]': 'цев',
 'word[-2:]': 'ев',
 'word.isdigit()': False}

In [755]:
len(X)

999

In [756]:
#разбиваем данные на train и test
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [757]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|███████████████████████████████████████████| 700/700 [00:00<00:00, 1118.17it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 36201
Seconds required: 0.161

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.08  loss=121565.02 active=36028 feature_norm=1.00
Iter 2   time=0.04  loss=109123.73 active=35080 feature_norm=1.60
Iter 3   time=0.04  loss=106856.31 active=35907 feature_norm=1.47
Iter 4   time=0.04  loss=105891.97 active=35865 feature_norm=1.41
Iter 5   time=0.04  loss=104524.09 active=35926 feature_norm=1.45
Iter 6   time=0.04  loss=91351.58 active=35827 feature_norm=4.28
Iter 7   time=0.04  loss=89575.26 active=36102 feature_norm=4.60
Iter 8   time=0.04  loss=86633.70 active=36186 feature_norm=4.98
Iter 9   time=0.04  loss=85236.06 active=36160 feature_norm=6.24
Iter 10 

In [758]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [759]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9408612727110087

In [760]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

              precision    recall  f1-score   support

         LOC       0.92      0.76      0.84      2613
         ORG       0.82      0.55      0.66      4260
         OUT       0.95      0.99      0.97     66116
         PER       0.92      0.79      0.85      4740

    accuracy                           0.94     77729
   macro avg       0.90      0.77      0.83     77729
weighted avg       0.94      0.94      0.94     77729



**Вывод:** передача в модель токенов и его соседей дает чуть более лучший результат и работает чуть дольше.

### 2.5. Выводы <a id='section_2.5'></a>

Готовые решения NER из spacy, slovnet показывают хорошие результаты в задачах извлечения именованных сущностей (NER из nltk справляется с задачей хуже). При составлении собственного алгоритма NER необходимо обращать внимание на контекст: передавать в алгоритм не только токен, но и его ближайших соседей (например, предыдущий и последующий токены).